# Setup

## Install packages
- `firebase-admin` will be used to publish TFLite model to firebase

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
!pip3 install {USER_FLAG} google-cloud-aiplatform --upgrade
!pip3 install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade
!pip3 install {USER_FLAG} firebase-admin

## Restart Jupyter Notebook programatically
- in order to reflect the package installations

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Check KFP(KubeFlow Pipeline) version

In [1]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.6.6


## Setup GCP Project ID

In [2]:
import os

PROJECT_ID = "grounded-atrium-320207"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  grounded-atrium-320207


## GCP Authentication

In [4]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

## Setup GCS Bucket name
- this bucket is where everything is going to stored

In [5]:
BUCKET_NAME = "gs://vertexai_dual_example"
REGION      = "us-central1"  

### Create GCS Bucket

In [6]:
!gsutil mb -l $REGION $BUCKET_NAME

Creating gs://vertexai_dual_example/...


## Setup GCS Path for Pipeline
- the pipeline runs are going to be stored (i.e. Metadata)

In [8]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

USER = "chansung"
PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)
PIPELINE_ROOT

env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://vertexai_dual_example/pipeline_root/chansung'

## Build Pipeline

### Install packages
- KFP related
- Client API for AI Platform

In [95]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from kfp.v2 import dsl
from kfp.v2.dsl import component

### Define pipeline component to publish TFLite model to Firebase
0. please follow the steps described from `Before you begin` section in the official [Deploy and manage custom models with Firebase Admin SDK] documentation. 
1. download credentials for the Firebase project
2. download TFLite model file
3. initialize firebase admin
4. upload and publish TFLite model from local file to Firebase

In [253]:
@component(packages_to_install=["google-cloud-storage", "firebase-admin", "tensorflow"])
def push_to_firebase():
    import firebase_admin
    from firebase_admin import ml
    from firebase_admin import storage
    from firebase_admin import credentials
    
    from google.cloud import storage as gcs_storage
    gcs_client = gcs_storage.Client()
    blobs = gcs_client.get_bucket("firebase-ml-bucket-gde-csp").list_blobs()
    for blob in blobs:
        if blob.name == "grounded-atrium-320207-firebase-adminsdk-5n9sn-20dbda9947.json":
            blob.download_to_filename('credential.json')
            break;
    
    target_blob = sorted(gcs_client.get_bucket("output-model-gde-csp").list_blobs(), reverse=True, key=lambda blob: blob.name.split('/')[-2])[0]
    target_blob.download_to_filename('model.tflite')
            
            
    firebase_admin.initialize_app(
        credentials.Certificate("credential.json"),
        options={
            "storageBucket": "grounded-atrium-320207.appspot.com"
        }
    )
    
    # Load a tflite file and upload it to Cloud Storage
    source = ml.TFLiteGCSModelSource.from_tflite_model_file('model.tflite')

    # Create the model object
    tflite_format = ml.TFLiteFormat(model_source=source)
    model = ml.Model(
        display_name="example_model",  # This is the name you use from your app to load the model.
        tags=["examples"],             # Optional tags for easier management.
        model_format=tflite_format)

    # Add the model to your Firebase project and publish it
    new_model = ml.create_model(model)
    ml.publish_model(new_model.model_id)    

### Define pipeline itself
1. Create Vertex AI's managed dataset from CSV
2. Define configs, one for cloud model, and the other one for mobile model
3. Run parallel processing for two different workflow(each workflow is configured appropriate for each target environment)
  - AutoML training can be configured differently for each target environment depending on `model_type`
4. Deploying the trained model as well as creating an endpoint is done with `ModelDeployOp` for cloud model
5. Export the trained mobile model to a GCS bucket
  - publish the exported mobile model to Firebase through push_to_firebase component
  

In [254]:
@kfp.dsl.pipeline(name="cloud-mobile-dual-deployment")
def pipeline(project: str = PROJECT_ID):
    ds_op = gcc_aip.ImageDatasetCreateOp(
       project=project,
       display_name="flowers-dataset",
       gcs_source="gs://dataset-meta-gde-csp/flowers_vertex.csv",
       import_schema_uri=aiplatform.schema.dataset.ioformat.image.multi_label_classification,
    )

    configs = [
       {
          "type": "CLOUD",
          "model_type": "CLOUD",
          "display_name": "train-cloud-model",
          "model_display_name": "cloud-model",
          "budget_milli_node_hours": 8000,
       },
       {
          "type": "MOBILE",
          "model_type": "MOBILE_TF_VERSATILE_1",
          "display_name": "train-mobile-model",
          "model_display_name": "mobile-model",
          "budget_milli_node_hours": 1000,
       }
    ]

    with kfp.dsl.ParallelFor(configs) as config:
        training_job_run_op = gcc_aip.AutoMLImageTrainingJobRunOp(
            project=project,
            display_name=config.display_name,
            prediction_type="classification",
            multi_label=True,
            model_type=config.model_type,
            base_model=None,
            dataset=ds_op.outputs["dataset"],
            model_display_name=config.model_display_name,
            training_fraction_split=0.6,
            validation_fraction_split=0.2,
            test_fraction_split=0.2,
            budget_milli_node_hours=config.budget_milli_node_hours,
        )

        with kfp.dsl.Condition(config.type=='CLOUD'):
            endpoint_op = gcc_aip.ModelDeployOp(
                project=project,
                model=training_job_run_op.outputs["model"]
            )

        with kfp.dsl.Condition(config.type=='MOBILE'):
            endpoint_op = gcc_aip.ModelExportOp( 
                project=project,
                model=training_job_run_op.outputs["model"],
                # tflite, edgetpu-tflite, tf-saved-model, tf-js, core-ml, custom-trained
                export_format_id="tflite",
                artifact_destination="gs://output-model-gde-csp/flower-models/"
            )

            push_to_firebase_task = push_to_firebase()
            

### Compile the pipeline
- you will get a json file for the pipeline spec after compiling.
  - you will only need this json file to run the pipeline

In [255]:
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="cloud-mobile-dual-deployment.json"
)

## Run the pipeline on Vertex AI Pipeline

### Create client instance to AI Platform (which is Vertex AI)

In [256]:
from kfp.v2.google.client import AIPlatformClient

api_client = AIPlatformClient(project_id=PROJECT_ID, region=REGION)

### Run the pipeline with the pipeline spec (json file)

In [257]:
response = api_client.create_run_from_job_spec(
    "cloud-mobile-dual-deployment.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"project": PROJECT_ID},
)